# Pyspark_Regression_Rdd_Housing

In [1]:
#1 - import module
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.ml import Pipeline
from pyspark.ml.feature import ChiSqSelector
import numpy as np
import pandas
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
#2 - Create SparkContext
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [4]:
sc._conf.getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.history.fs.cleaner.interval', u'7d'),
 (u'spark.shuffle.io.serverThreads', u'128'),
 (u'spark.yarn.historyServer.address',
  u'hdp-all-spark-new.c.kao-project-v1.internal:18081'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  u'hdp-all-spark-new.c.kao-project-v1.internal'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.driver.port', u'39956'),
 (u'spark.driver.host', u'hdp-all-spark-new.c.kao-project-v1.internal'),
 (u'spark.sql.statistics.fallBackToHdfs', u'true'),
 (u'spark.shuffle.file.buffer', u'1m'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.his

In [5]:
print sc.getConf().toDebugString()

spark.app.id=application_1541652388947_0017
spark.app.name=pyspark-shell
spark.driver.appUIAddress=http://hdp-all-spark-new.c.kao-project-v1.internal:4040
spark.driver.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64
spark.driver.host=hdp-all-spark-new.c.kao-project-v1.internal
spark.driver.port=39956
spark.eventLog.dir=hdfs:///spark2-history/
spark.eventLog.enabled=true
spark.executor.extraJavaOptions=-XX:+UseNUMA
spark.executor.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64
spark.executor.id=driver
spark.executorEnv.PYTHONPATH=/usr/hdp/3.0.0.0-1634/spark2/python:/usr/hdp/3.0.0.0-1634/spark2/python/lib/py4j-0.10.7-src.zip:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip
spark.history.fs.cleaner.enabled=true
spark.history.fs.cleaner.interval=7d
spark.history.fs.cleaner.maxAge=90d
spark.history.fs.logDirectory=hdfs:///spark2-history/
spark.histo

In [6]:
#3 - Setup SparkSession(SparkSQL)
spark = (SparkSession
         .builder
         .appName("Pyspark_Regression_Rdd_Housing")
         .getOrCreate())
print spark

In [1]:
!hdfs dfs -cat data/housing.txt


























































































































































































































































































































































































































































































































11.9 1:0.04741 2:0.00 3:11.930 4:0 5:0.5730 6:6.0300 7:80.80 8:2.5050 9:1 10:273.0 11:21.00 12:396.90 13:7.88

In [7]:
#4 - Read file to spark DataFrame

data = (spark
        .read
        .format('libsvm')
        .load("data/housing.txt"))
data.cache()
print "finish caching data"

finish caching data


In [8]:
data.describe().toPandas()

summary               label
0   count                 506
1    mean  22.532806324110698
2  stddev   9.197104087379815
3     min                 5.0
4     max                50.0

In [9]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 24.0|(13,[0,1,2,4,5,6,...|
| 21.6|(13,[0,2,4,5,6,7,...|
| 34.7|(13,[0,2,4,5,6,7,...|
| 33.4|(13,[0,2,4,5,6,7,...|
| 36.2|(13,[0,2,4,5,6,7,...|
+-----+--------------------+
only showing top 5 rows



In [13]:
#5 - Print sample 5 rows of all variables
print "number of features : " + str((data.first()[1]).size)

number of features : 13


In [14]:
#6 - split Train and Test data
data = data.sort('label')
(trainingData, testData) = data.randomSplit([0.7, 0.3],seed = 50)

print "data count : " + str(data.count())
print "trainingData count : " + str(trainingData.count())
print "testData count : " + str(testData.count())


data count : 506
trainingData count : 360
testData count : 146


In [15]:
#7 - Create model
regr  = LinearRegression(maxIter= 100,labelCol="label", featuresCol="features")

In [16]:
#8 - Set ML pipeline
all_process_list = [regr]
for process in all_process_list: print process

pipeline = Pipeline(stages=all_process_list)
print pipeline

LinearRegression_42ffa085a2f02a7de710
Pipeline_4f85becafc6ccbc456c6


In [17]:
#9 - Train model
model = pipeline.fit(trainingData)

In [18]:
#10 - (Optional) Assign multiple parameter lists used to train multiple models
# Train model and vary parameter using crossvalidation
paramGrid = (ParamGridBuilder()
    .addGrid(regr.elasticNetParam, [0.0,0.5,1.0])        
    .build())

#elasticNetParam
#    For alpha = 0, the penalty is an L2 penalty. 
#    For alpha = 1, it is an L1 penalty.'

print len(paramGrid)

#for param in paramGrid: 
#    print str(param) + "\n"

3


In [ ]:
#11 - (Optional) Train multiple models with multiple parameters
crossval = CrossValidator(estimator=pipeline,
                      estimatorParamMaps=paramGrid,
                      evaluator=RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse"),
                      numFolds=3)
cvModel = crossval.fit(trainingData)
model = cvModel.bestModel

print model

In [19]:
#12 - Make predictions
predictions = model.transform(testData)
predictions.cache()

DataFrame[label: double, features: vector, prediction: double]

In [20]:
# Print sample result
predictions.toPandas()

label                                           features  prediction
0      5.0  (67.9208, 0.0, 18.1, 0.0, 0.693, 5.683, 100.0,...    9.288211
1      6.3  (9.91655, 0.0, 18.1, 0.0, 0.693, 5.852, 77.8, ...   10.148429
2      7.5  (10.8342, 0.0, 18.1, 0.0, 0.679, 6.782, 90.8, ...   12.194524
3      8.4  (13.6781, 0.0, 18.1, 0.0, 0.74, 5.935, 87.9, 1...    3.550438
4      8.5  (7.67202, 0.0, 18.1, 0.0, 0.693, 5.747, 98.9, ...   16.645593
5      8.7  (15.1772, 0.0, 18.1, 0.0, 0.74, 6.152, 100.0, ...    7.956898
6      8.8  (20.0849, 0.0, 18.1, 0.0, 0.7, 4.368, 91.2, 1....    3.302378
7      9.6  (14.4208, 0.0, 18.1, 0.0, 0.74, 6.461, 93.3, 2...   13.855136
8     10.2  (17.8667, 0.0, 18.1, 0.0, 0.671, 6.223, 100.0,...   17.099576
9     10.4  (25.9406, 0.0, 18.1, 0.0, 0.679, 5.304, 89.1, ...    6.567214
10    10.8  (12.8023, 0.0, 18.1, 0.0, 0.74, 5.854, 96.6, 1...   11.248026
11    10.9  (37.6619, 0.0, 18.1, 0.0, 0.679, 6.202, 78.7, ...   13.785085
12    11.3  (9.18702, 0.0, 18.1, 0.0, 0.7, 5.536, 100.0, 1...   13.666494
13    11.9  (0.04741, 0.0, 11.93, 0.0, 0.573, 6.03, 80.8, ...   22.719010
14    12.1  (9.59571, 0.0, 18.1, 0.0, 0.693, 6.404, 100.0,...   18.352888
15    12.3  (7.99248, 0.0, 18.1, 0.0, 0.7, 5.52, 100.0, 1....   13.239145
16    13.1  (1.15172, 0.0, 8.14, 0.0, 0.538, 5.701, 95.0, ...   14.438008
17    13.1  (2.44668, 0.0, 19.58, 0.0, 0.871, 5.272, 94.0,...   13.758314
18    13.1  (23.6482, 0.0, 18.1, 0.0, 0.671, 6.38, 96.2, 1...   15.966508
19    13.2  (1.38799, 0.0, 8.14, 0.0, 0.538, 5.95, 82.0, 3...    7.909804
20    13.3  (6.39312, 0.0, 18.1, 0.0, 0.584, 6.162, 97.4, ...   16.052823
21    13.4  (3.32105, 0.0, 19.58, 1.0, 0.871, 5.403, 100.0...   14.471455
22    13.4  (6.71772, 0.0, 18.1, 0.0, 0.713, 6.749, 92.6, ...   15.666428
23    13.4  (11.1604, 0.0, 18.1, 0.0, 0.74, 6.629, 94.6, 2...   12.540476
24    13.5  (8.20058, 0.0, 18.1, 0.0, 0.713, 5.936, 80.3, ...   12.103194
25    13.8  (8.64476, 0.0, 18.1, 0.0, 0.693, 6.193, 92.6, ...   20.512883
26    14.3  (5.58107, 0.0, 18.1, 0.0, 0.713, 6.436, 87.9, ...   16.427286
27    14.9  (6.28807, 0.0, 18.1, 0.0, 0.74, 6.341, 96.4, 2...   17.615155
28    15.1  (6.96215, 0.0, 18.1, 0.0, 0.7, 5.713, 97.0, 1....   17.623458
29    15.2  (1.23247, 0.0, 8.14, 0.0, 0.538, 6.142, 91.7, ...   15.844720
..     ...                                                ...         ...
116   29.0  (0.05561, 70.0, 2.24, 0.0, 0.4, 7.041, 10.0, 7...   31.531425
117   29.4  (0.06664, 0.0, 4.05, 0.0, 0.51, 6.546, 33.1, 3...   30.195156
118   29.6  (0.06047, 0.0, 2.46, 0.0, 0.488, 6.153, 68.8, ...   24.661165
119   30.1  (0.65665, 20.0, 3.97, 0.0, 0.647, 6.842, 100.0...   35.834775
120   31.0  (0.82526, 20.0, 3.97, 0.0, 0.647, 7.327, 94.5,...   34.813928
121   31.2  (0.03049, 55.0, 3.78, 0.0, 0.484, 6.874, 28.1,...   28.191687
122   31.5  (0.51183, 0.0, 6.2, 0.0, 0.507, 7.358, 71.6, 4...   32.851060
123   31.6  (0.41238, 0.0, 6.2, 0.0, 0.504, 7.163, 79.9, 3...   32.613074
124   31.7  (0.46296, 0.0, 6.2, 0.0, 0.504, 7.412, 76.9, 3...   33.441320
125   32.5  (0.10008, 0.0, 2.46, 0.0, 0.488, 6.563, 95.6, ...   31.678053
126   32.7  (0.01301, 35.0, 1.52, 0.0, 0.442, 7.241, 49.3,...   30.363106
127   33.0  (0.01951, 17.5, 1.38, 0.0, 0.4161, 7.104, 59.5...   23.382285
128   33.1  (0.1, 34.0, 6.09, 0.0, 0.433, 6.982, 17.7, 5.4...   32.364003
129   33.2  (0.0686, 0.0, 2.89, 0.0, 0.445, 7.416, 62.5, 3...   31.987080
130   33.3  (0.04011, 80.0, 1.52, 0.0, 0.404, 7.287, 34.1,...   36.431771
131   33.4  (0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, ...   28.329621
132   34.9  (0.0837, 45.0, 3.44, 0.0, 0.437, 7.185, 38.9, ...   34.363704
133   35.4  (0.03705, 20.0, 3.33, 0.0, 0.4429, 6.968, 37.2...   34.261530
134   36.4  (0.08664, 45.0, 3.44, 0.0, 0.437, 7.178, 26.3,...   32.703110
135   36.5  (0.55007, 20.0, 3.97, 0.0, 0.647, 7.206, 91.6,...   36.311539
136   37.2  (0.0578, 0.0, 2.46, 0.0, 0.488, 6.98, 58.4, 2....   32.721968
137   37.6  (0.38214, 0.0, 6.2, 0.0, 0.504, 8.04, 86.5, 3....   37.838154
138   38.

In [21]:
#13 - Evaluate model on training data (not test data)
regrModel = model.stages[-1]
trainingSummary = regrModel.summary
print("Coefficients: %s" % str(regrModel.coefficients))
print("Intercept: %s" % str(regrModel.intercept))
print ("RMSE: %f" % regrModel.summary.rootMeanSquaredError)
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
print("r2: %f" % trainingSummary.r2)
trainingSummary.residuals.toPandas()

Coefficients: [-0.09309174416482018,0.051812000173758824,0.06330332447607223,2.2339400241737395,-19.437874970851777,3.481634001278956,0.02162866199685257,-1.4298977373048578,0.3102154998647826,-0.012948157257467187,-1.0333199413797538,0.011372103317816695,-0.573414654400564]
Intercept: 38.8754317003
RMSE: 4.734426
numIterations: 1
objectiveHistory: [0.0]
r2: 0.736214


residuals
0    -1.920249
1    -6.466691
2    -1.675166
3    11.434918
4   -10.832009
5    -0.900166
6    -1.480474
7     1.797426
8     4.212810
9    -0.957388
10   -5.332540
11   -4.873392
12    1.014467
13    2.185189
14   -2.751057
15   -0.474408
16   -5.318638
17    3.607658
18   -5.111466
19   -2.392627
20    3.848613
21   -7.733670
22   -2.329112
23   -3.096454
24   -2.253061
25   -4.052293
26    2.751574
27    0.772649
28    2.956221
29    0.105718
..         ...
330   8.505830
331   8.517996
332   6.615806
333   3.092036
334   3.562655
335   5.141561
336   6.819482
337   3.718254
338   5.583320
339   5.554048
340   4.199155
341   6.300542
342   6.492352
343   6.375110
344   6.098867
345  12.382887
346  11.332754
347   6.450319
348   5.861274
349   9.281750
350   6.666391
351   7.797371
352   8.739226
353  17.038402
354  23.908794
355  14.658406
356  10.327984
357  11.849760
358  24.789056
359  24.322628

[360 rows x 1 columns]

In [ ]:
#14 - Evaluate model on test data

print "rmse (default): root mean squared error"
print "mse: mean squared error"
print "r2: R2 metric"
print "mae: mean absolute error"

for metricName in ['rmse','mse','r2','mae']:
    evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName=metricName)
    result = evaluator.evaluate(predictions)
    print '%s = %g' % (metricName,result)
